In [ ]:
import time
import numpy as np 
from zmqRemoteApi import RemoteAPIClient

In [ ]:
def degree_To_radian(degree):
    return (degree * np.pi/180)

def FK(theta, alfa, a, d):
    
    Cos_theta = np.cos(theta)
    Sin_theta = np.sin(theta)
    Cos_alfa = np.cos(alfa)
    Sin_alfa = np.sin(alfa)
    Transformation_mat = np.array([
                                    [Cos_theta,-Sin_theta*Cos_alfa,Sin_theta*Sin_alfa,a*Cos_theta],
                                    [Sin_theta,Cos_theta*Cos_alfa,-Cos_theta*Sin_alfa,a*Sin_theta],
                                    [0,Sin_alfa,Cos_alfa,d],
                                    [0,0,0,1]
                                  ])
    return np.round(Transformation_mat.reshape(4,4),2)

In [ ]:
def Calculate_Jacopian_P(Trans_mat_arr):
    
    O_0 = np.zeros([3,1])
    Jp_size = len(Trans_mat_arr)
    Z_0 = np.array([[0,0,1]]).reshape(3,1)
    Z_arr = [z[0:3,2] for z in Trans_mat_arr]
    Z_arr.insert(0,Z_0)
    O_arr = [o[0:3,3] for o in Trans_mat_arr]
    O_arr.insert(0,O_0)
    O_last = O_arr[-1].reshape(3,1)
    jp_mat = np.array([
        [np.cross(Z_arr[i].reshape(3,1) , (O_last-O_arr[i].reshape(3,1)),axis=0) for i in range(Jp_size)]
    ])
    return jp_mat.reshape(3,Jp_size).transpose()
    
def Calculate_Jacopian_r(Trans_mat_arr):
    
    Z_0 = np.array([[0,0,1]])
    Jp_size = len(Trans_mat_arr)
    Z_arr = [Trans_mat_arr[z][0:3,2] for z in range(Jp_size-1)]
    Z_arr.insert(0,Z_0)
    Jr_mat = np.array([[z.reshape(3,1) for z in Z_arr]])
    return Jr_mat.reshape(3,Jp_size).transpose()

def Calculate_J(Trans_mat_arr):
    
    J_size = len(Trans_mat_arr)
    J = np.array([
                    [Calculate_Jacopian_P(Trans_mat_arr),
                     Calculate_Jacopian_r(Trans_mat_arr)]
                ])
    return J.reshape(6,J_size)

In [ ]:
def Calculate_0n_Matrix(Trans_mat_arr):
    
    n = len(Trans_mat_arr)
    T_0 = np.identity(4)
    result_mat = []
    for i in range(n):
        T_0 = np.matmul(T_0,Trans_mat_arr[i])
        T_0 = np.round(T_0,2)
        result_mat.append( T_0 )
    return result_mat

In [ ]:
def Calculate_Delta_X(Td, T0):
    
    Pd = Td[0:3,3]
    P0 = T0[0:3,3]
    Rd = Td[0:3,0:3]
    R0 = T0[0:3,0:3]
    I = np.identity(3)
    Pos_list = np.array([[d-c for d,c in zip(Pd,P0)]])
    Pos_list = Pos_list.reshape(3,1)
    DR = np.matmul(Rd,R0.transpose())
    skew = np.subtract(DR , I)
    skew = np.round(skew,2)
    vec = np.array([[abs(skew[1,2]),abs(skew[0,2]) ,abs(skew[0,1])]])
    vec = vec.reshape(3,1)
    Delta_X = np.array([Pos_list,vec])
    return np.round(Delta_X.reshape(6,1),2)  

In [ ]:
def Calculate_IK(dX,J,q0):
    # qd = q0 + J+ * d_X
    j_pinv = np.linalg.pinv(J)
    q0_arr = np.array([q0])
    q0_arr = q0_arr.reshape(6,1)
    qd = np.add(q0_arr,np.matmul(np.round(j_pinv,2),dX)) # 6*1
    return np.round(qd.reshape(1,6),2)

In [ ]:
client = RemoteAPIClient()
sim = client.getObject('sim')
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)
sim.setInt32Param(sim.intparam_idle_fps, 0)

In [ ]:
# set Object name 
joint_names = [f"joint_{i+1}"  for i in range(6)]
joint_handels = [sim.getObjectHandle(joint) for joint in joint_names]
joint_Position = [sim.getJointPosition(handle) for handle in joint_handels]
for handle in joint_handels:
    sim.setObjectInt32Parameter(handle ,2001,~0) #to enable the position control

In [ ]:
q0 = [0,0,0,0,0,0]
alpha = [90,0,0,90,-90,0]
D = [0.1519,0,0,0.11235,0.08535,0.0819]
A = [0,-0.24365,-0.21325,0,0,0]

# calc Transformation matrix using FK
Trans_list = [FK(degree_To_radian(q0[i]),degree_To_radian(alpha[i]),A[i],D[i]) for i in range(6)]
T_0n = Calculate_0n_Matrix(Trans_list)
# calc T disierd
T_d = FK(degree_To_radian(45),degree_To_radian(45),0,0)
# claculate Jacopian matrix
J = Calculate_J(T_0n)
# calculate delta X matrix  
delta_X = Calculate_Delta_X(T_d,T_0n[-1])
# calculate qd using IK 
qd = Calculate_IK(delta_X,J,q0)

In [ ]:
while ~( np.isclose( np.linalg.norm(delta_X),0.9 ,rtol=0.08)  ):
    
    q0 = qd.copy()
    Trans_list.clear()
    Trans_list = [FK(q0[0][i],degree_To_radian(alpha[i]),A[i],D[i]) for i in range(6)]
    T_0n = Calculate_0n_Matrix(Trans_list)
    J = Calculate_J(T_0n)
    delta_X = Calculate_Delta_X(T_d,T_0n[-1])
    qd = Calculate_IK(delta_X,J,q0)

In [ ]:
client.setStepping(False)
sim.startSimulation()

for (handle,degree) in zip(joint_handels,qd[0]):
    sim.startSimulation()
    sim.setJointTargetPosition(handle, degree_To_radian(degree) ) # degree by radian
    sim.startSimulation()

In [ ]:
sim.stopSimulation()